In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import transforms, datasets
import numpy as np
import os
import matplotlib.pyplot as plt

In [3]:
os.chdir('data')

In [4]:
data_path = os.getcwd()

In [5]:
train_data = datasets.CIFAR10(data_path, train=True, download = True, 
                              transform =  transforms.ToTensor())

Files already downloaded and verified


In [6]:
test_data = datasets.CIFAR10(data_path, train=False, download = True, 
                              transform =  transforms.ToTensor())

Files already downloaded and verified


In [7]:
train_loader = torch.utils.data.DataLoader(train_data, batch_size = 32, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size = 32, shuffle=True)

In [46]:
class Net(nn.Module):
    
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=(3, 3), padding=1)
        self.conv2 = nn.Conv2d(32, 32, kernel_size=(3, 3), padding=1)
        self.bn = nn.BatchNorm2d(num_features=32)
        self.fc1 = nn.Linear(32*4*4, 256)
        self.fc2 = nn.Linear(256, 10)
    
    def forward(self, x):
        out = self.bn(self.conv1(x))
        out = F.max_pool2d(torch.relu(out), 2)
        out = self.bn(self.conv2(out))
        out = F.max_pool2d(torch.relu(out), 2)
        out = self.bn(self.conv2(out))
        out = F.max_pool2d(torch.relu(out), 2)
        out = out.reshape(-1, 32*4*4)
        out = F.relu(self.fc1(out))
        return self.fc2(out)

In [47]:
model = Net().to('cuda')

In [64]:
_, preds = torch.max(model(img.to('cuda')), dim=1)
preds

tensor([5, 0, 9, 8, 4, 2, 9, 5, 1, 2, 8, 1, 2, 1, 2, 5, 1, 2, 3, 5, 7, 1, 2, 5,
        5, 9, 3, 5, 1, 3, 1, 2, 1, 8, 4, 7, 8, 5, 3, 2, 5, 9, 9, 9, 8, 9, 4, 4,
        7, 7, 4, 5, 0, 5, 4, 9, 9, 9, 9, 3, 0, 5, 6, 1], device='cuda:0')

In [68]:
(preds == label.to('cuda')).sum()

tensor(53, device='cuda:0')

In [69]:
label.size(0)

64

In [8]:
def training_loop(n_epochs, optimizer, model, loss_fn, train_loader):
    for epoch in range(1, n_epochs+1):
        loss_sum = 0.0
        num_samples = 0.0
        
        for imgs, labels in train_loader:
            imgs = imgs.to('cuda')
            labels = labels.to('cuda')
            
            scores = model(imgs)
            
            loss = loss_fn(scores, labels)
            
            optimizer.zero_grad()
            
            loss.backward()
            
            optimizer.step()
            
            loss_sum += loss.item()
            num_samples += labels.size(0)
        if epoch == 1 or epoch % 5 == 0:
            print(f'epoch: {epoch} \t loss: {loss_sum/num_samples}')

In [14]:
loss_fn = nn.CrossEntropyLoss()

In [50]:
optimizer = optim.Adam(model.parameters(), lr=1e-2)

training_loop(n_epochs = 50,
              optimizer = optimizer,
              model = model,
              loss_fn = loss_fn,
              train_loader= train_loader)

epoch: 1 	 loss: 0.02317188651561737
epoch: 5 	 loss: 0.013153576763868332
epoch: 10 	 loss: 0.01128093208372593
epoch: 15 	 loss: 0.010249423859119415
epoch: 20 	 loss: 0.00948134977698326
epoch: 25 	 loss: 0.009088031480312347
epoch: 30 	 loss: 0.008768470445275307
epoch: 35 	 loss: 0.00837251791447401
epoch: 40 	 loss: 0.008123035531938076
epoch: 45 	 loss: 0.007983841491937637
epoch: 50 	 loss: 0.007764490563571453


In [9]:
def test_loop(model, test_loader):
    num_correct = 0
    num_samples = 0
    for imgs, labels in test_loader:
        
        imgs = imgs.to('cuda')
        labels = labels.to('cuda')
        
        _, preds = torch.max(model(imgs), dim=1)
        
        num_correct += (preds == labels).sum()
        num_samples += labels.size(0)
    print(f"correct_predictions: {num_correct} / {num_samples} \t accuracy: {float(num_correct/num_samples)}")

In [73]:
test_loop(model, test_loader)

correct_predictions: 7086 / 10000 	 accuracy: 0.7085999846458435


In [ ]:
class ResDense(nn.Module):
    def __init__(self, neurons):
        super(ResDense, self).__init__()
        
        self.dense = nn.Linear(neurons, neurons)
        self.bn = nn.BatchNorm1d(neurons)
    def forward(self, x):
        out = torch.relu(s)

In [10]:
class ResBlock(nn.Module):
    def __init__(self, filters):
        super(ResBlock, self).__init__()
        
        self.conv = nn.Conv2d(filters, filters, kernel_size=(5, 5), padding=2)
        self.bn = nn.BatchNorm2d(num_features=filters)
    
    def forward(self, x):
        out = torch.relu(self.bn(self.conv(x)))
        return (out + x)

In [11]:
class ResNet(nn.Module):
    def __init__(self, num_filters, num_blocks):
        super(ResNet, self).__init__()
        self.num_filters = num_filters
        self.conv1 = nn.Conv2d(3, num_filters, kernel_size=(5, 5), padding=2)
        self.bn1 = nn.BatchNorm2d(num_features=num_filters)
        self.res = nn.Sequential( * ( num_blocks * [ResBlock(num_filters)]))
        
        self.conv_memory = nn.Conv2d(3, num_filters, kernel_size=(1, 1))
        
        self.memory1 = nn.Sequential(self.conv_memory,
                                     nn.MaxPool2d(2))
        
        self.memory2 = nn.Sequential(self.conv_memory,
                                     nn.MaxPool2d(4))
        
        self.fc1 = nn.Linear(num_filters * 4 * 4, 64)
        self.fc2 = nn.Linear(64, 10)
    
    def forward(self, x):
        out = torch.relu(self.bn1(self.conv1(x)))
        out = F.max_pool2d((self.res(out) + self.conv_memory(x)), 2)
        out = F.max_pool2d((self.res(out) + self.memory1(x)), 2)
        out = F.max_pool2d((self.res(out) + self.memory2(x)), 2)
        out = out.reshape(-1, self.num_filters * 4 *4)
        out = torch.relu(self.fc1(out))
        out = self.fc2(out)
        return out

In [12]:
res_model = ResNet(8, 3).to('cuda')

In [15]:
res_optimizer = optim.Adam(res_model.parameters(), lr=1e-3)

training_loop(n_epochs = 50,
              optimizer = res_optimizer,
              model = res_model,
              loss_fn = loss_fn,
              train_loader= train_loader)

C:\Users\An_asus\Anaconda3\lib\site-packages\torch\nn\functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ..\c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


epoch: 1 	 loss: 0.05122611243605614
epoch: 5 	 loss: 0.035530791873931884
epoch: 10 	 loss: 0.031444659615159035
epoch: 15 	 loss: 0.029040825434923173
epoch: 20 	 loss: 0.027298846329450607
epoch: 25 	 loss: 0.026026308337450028
epoch: 30 	 loss: 0.025165295752882958
epoch: 35 	 loss: 0.024436747311353684
epoch: 40 	 loss: 0.023795349410772323
epoch: 45 	 loss: 0.023388135784864424
epoch: 50 	 loss: 0.02284540410399437


In [16]:
test_loop(model=res_model, test_loader = test_loader)

correct_predictions: 6808 / 10000 	 accuracy: 0.6807999610900879


In [17]:
class ResConv(nn.Module):
    def __init__(self, filters):
        super(ResConv, self).__init__()
        
        self.conv = nn.Conv2d(filters, filters, kernel_size=(3, 3), padding=1)
    
    def forward(self, x):
        out = torch.relu(self.conv(x))
        return (out + x)

In [18]:
class MemoryNet(nn.Module):
    def __init__(self, num_filters, num_blocks):
        super(MemoryNet, self).__init__()
        self.num_filters = num_filters
        
        self.conv1 = nn.Conv2d(3, num_filters, kernel_size=(3, 3), padding=1)

        self.res = nn.Sequential( * ( num_blocks * [ResConv(num_filters)]))
        
        self.conv_memory = nn.Conv2d(3, num_filters, kernel_size=(1, 1))
        
        self.memory1 = nn.Sequential(self.conv_memory,
                                     nn.MaxPool2d(2))
        
        self.memory2 = nn.Sequential(self.conv_memory,
                                     nn.MaxPool2d(4))
        
        self.fc1 = nn.Linear(num_filters * 4 * 4, 64)
        self.fc2 = nn.Linear(64, 10)
    
    def forward(self, x):
        out = torch.relu(self.conv1(x))
        out = F.max_pool2d((self.res(out) + self.conv_memory(x)), 2)
        out = F.max_pool2d((self.res(out) + self.memory1(x)), 2)
        out = F.max_pool2d((self.res(out) + self.memory2(x)), 2)
        out = out.reshape(-1, self.num_filters * 4 *4)
        out = torch.relu(self.fc1(out))
        out = self.fc2(out)
        return out

In [19]:
mem_model = MemoryNet(16, 3).to('cuda')

mem_optimizer = optim.Adam(mem_model.parameters(), lr=1e-3)

training_loop(n_epochs = 50,
              optimizer = mem_optimizer,
              model = mem_model,
              loss_fn = loss_fn,
              train_loader= train_loader)

epoch: 1 	 loss: 0.05032935439944267
epoch: 5 	 loss: 0.03129648393690586
epoch: 10 	 loss: 0.026305621837377547
epoch: 15 	 loss: 0.023850160502195357
epoch: 20 	 loss: 0.022230680535435678
epoch: 25 	 loss: 0.02104149684280157
epoch: 30 	 loss: 0.020186014511287213
epoch: 35 	 loss: 0.019479490060806275
epoch: 40 	 loss: 0.01889821978479624
epoch: 45 	 loss: 0.018417731516957282
epoch: 50 	 loss: 0.01794390323251486


In [20]:
test_loop(model=mem_model, test_loader = test_loader)

correct_predictions: 6982 / 10000 	 accuracy: 0.698199987411499
